In [4]:
import json
from src.generator import GeneratorFactory
from src.prompts import CfgNetPromptSettings
from src.util import transform
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env")

print(f"Base URL: {os.getenv("BASE_URL")}")
print(f"Proxy Server Key: {os.getenv("PROXY_SERVER_API_KEY")}")


# TODO: define genrators
generator = GeneratorFactory().get_generator(model_name="gpt-4o-mini-2024-07-18", temperature=0.7)

with open("data/evaluation/retrieval_results/test_dependencies_config1.json", "r", encoding="utf-8") as src:
    data = json.load(src)

for sample in data:
    dependency = transform(entry=sample)

    # create prompt parts
    system_str = CfgNetPromptSettings.get_system_str(dependency=dependency)
    task_str = CfgNetPromptSettings.get_task_str(dependency=dependency)
    context_str = "\n\n".join([x["text"] for x in sample["context"]])
    format_str = CfgNetPromptSettings.get_format_prompt()

    query_str = CfgNetPromptSettings.query_prompt.format(
        context_str=context_str,
        task_str=task_str,
        format_str=format_str
    )

    messages = [
        {"role": "system", "content": system_str},
        {"role": "user", "content": query_str }
    ]

    response = generator.generate(messages=messages)

    print(response, type(response))

    sample["genrations"] = {generator.model_name: response} 



Base URL: http://172.26.92.115
Proxy Server Key: 0ad58c3e-b9f4-4913-b8c9-7cd8e2fec7ab
GPT (gpt-4o-mini-2024-07-18) generator initialized.
Start generation using gpt-4o-mini-2024-07-18


[02/05/25 17:15:07] INFO     HTTP Request: POST http://172.26.92.115/chat/completions "HTTP/1.1 200 ]8;id=250922;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=415583;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Add response to answer pool.
Start generation using gpt-4o-mini-2024-07-18


[02/05/25 17:15:11] INFO     HTTP Request: POST http://172.26.92.115/chat/completions "HTTP/1.1 200 ]8;id=565554;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=779334;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Add response to answer pool.
Start generation using gpt-4o-mini-2024-07-18


...

Add response to answer pool.
Start generation using gpt-4o-mini-2024-07-18


[02/05/25 17:15:51] INFO     HTTP Request: POST http://172.26.92.115/chat/completions "HTTP/1.1 200 ]8;id=646919;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=639373;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Add response to answer pool.
Start generation using gpt-4o-mini-2024-07-18


[02/05/25 17:15:55] INFO     HTTP Request: POST http://172.26.92.115/chat/completions "HTTP/1.1 200 ]8;id=534725;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=836185;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Add response to answer pool.
Get dominant response.
{'plan': '1. Identify the two configuration options: spring.cloud.zookeeper.enabled from apollo-assembly/src/main/resources/application.yml and eureka.client.fetch-registry from apollo-configservice/src/main/resources/application.yml. 2. Check the types of both configuration options to confirm they are BOOLEAN. 3. Verify the values of both options: spring.cloud.zookeeper.enabled has a value of FALSE and eureka.client.fetch-registry also has a value of FALSE. 4. Determine if the identical values (both FALSE) create a dependency based on value-equality. 5. Conclude whether a true dependency exists or if the similarity in values is coincidental.', 'rationale': 'Both configuration options are of BOOLEAN type and share the same value of FALSE. However, even though they have equal values, there is no indication from the context that these options directly depend on each other for functionality. Therefore, the equal values do not imply a nec

In [5]:
data

[{'dependency_category': 'value-equality',
  'index': 0,
  'link_str': 'apollo::::apollo-audit/apollo-audit-api/pom.xml::::project::::modelVersion::::modelVersion:4.0.0 <-> apollo::::apollo-biz/pom.xml::::project::::modelVersion::::modelVersion:4.0.0',
  'project': 'apollo',
  'option_name': 'project.modelVersion',
  'option_value': 'modelVersion:4.0.0',
  'option_type': 'ConfigType.VERSION_NUMBER',
  'option_file': 'apollo-audit/apollo-audit-api/pom.xml',
  'option_technology': 'maven',
  'dependent_option_name': 'project.modelVersion',
  'dependent_option_value': 'modelVersion:4.0.0',
  'dependent_option_type': 'ConfigType.VERSION_NUMBER',
  'dependent_option_file': 'apollo-biz/pom.xml',
  'dependent_option_technology': 'maven',
  'config_type': 'correct',
  'rating': True,
  'final_rating': True,
  'final_category': 'Version Numbers',
  'sub_category': 'POM Version',
  'explanation': 'modelVersion has to be consistent across all modules and is generally set to 4.0.0 since it is the 

In [1]:
import json
from src.util import compute_evaluation_metrics

with open("data/evaluation/generation_results/test_dependencies_config1.json", "r", encoding="utf-8") as src:
    data = json.load(src)

metrics = compute_evaluation_metrics(data)

for x in metrics:
    print(x)

/Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model:  gpt-4o-mini-2024-07-18
TP:  1
FP:  0
FN:  0
TN:  3
Model:  deepseek-r1:70b
TP:  0
FP:  0
FN:  1
TN:  3
Model:  phi4:latest
TP:  0
FP:  0
FN:  1
TN:  3
Model:  llama3.1:70b
TP:  0
FP:  0
FN:  1
TN:  3
{'model': 'gpt-4o-mini-2024-07-18', 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0, 'accuracy': 1.0, 'skipped': 0}
{'model': 'deepseek-r1:70b', 'precision': 0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.75, 'skipped': 0}
{'model': 'phi4:latest', 'precision': 0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.75, 'skipped': 0}
{'model': 'llama3.1:70b', 'precision': 0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.75, 'skipped': 0}


In [2]:
from src.util import get_dominat_response

responses = [
    {
    "plan": "1. Identify the configuration options in question: both are 'project.modelVersion' of type VERSION_NUMBER with value 4.0.0.\n2. Check if these configurations belong to different Maven modules (apollo-audit/apollo-audit-api and apollo-biz).\n3. Determine if there's a parent-child relationship between the POM files or if they are independent.\n4. Assess whether Maven requires modelVersion to be consistent across modules in a multi-module project.\n5. Conclude based on Maven's requirements for modelVersion consistency.",
    "rationale": "In Maven, each module's pom.xml must have the same modelVersion as specified in the parent POM if it's part of a multi-module project. If these modules share a common parent with a defined modelVersion, then their individual 'project.modelVersion' values must match to maintain consistency and avoid configuration errors.",
    "uncertainty": 8,
    "isDependency": True
    },
    {
    "plan": "1. Identify both configuration options in their respective POM files.\n2. Check if they are of the same type (VERSION_NUMBER) and have the same value (4.0.0).\n3. Determine if Maven requires these versions to be identical for proper functionality across dependent projects.\n4. Conclude based on Maven's handling of modelVersion in multi-module projects.",
    "rationale": "In Maven, the modelVersion typically refers to the version of the POM structure and is usually consistent across all modules within a multi-module project to ensure compatibility. If apollo-audit-api and apollo-biz are part of the same parent project or have interdependencies, having the same modelVersion ensures that Maven can correctly process and build them together without conflicts.",
    "uncertainty": 8,
    "isDependency": True
    },
    {
    "plan": "1. Identify the two configuration options in question: project.modelVersion in apollo-audit/apollo-audit-api/pom.xml with value 4.0.0 and project.modelVersion in apollo-biz/pom.xml also with value 4.0.0.\n2. Understand that both are Maven POM files, which use modelVersion to specify the version of the POM format they conform to.\n3. Recognize that modelVersion is a standard element in Maven POMs and typically refers to the Project Object Model version (e.g., 4.0.0).\n4. Note that having the same value doesn't inherently create a dependency; it's about whether their values must match for proper functionality.\n5. Consider whether these two projects are part of the same multi-module project or independent.\n6. If they're modules in a parent POM, the modelVersion should align with the parent's modelVersion to maintain consistency and avoid errors during build processes.\n7. However, if they are separate projects without a shared parent, their modelVersions can differ without causing dependency issues because each project is built independently.\n8. Without explicit information about a parent-child relationship or inter-dependency between these two POMs, it's reasonable to conclude that their modelVersion values don't depend on each other.\n9. Therefore, the same value might be coincidental and not indicative of an actual dependency.",
    "rationale": "The project.modelVersion in both POM files refers to the Maven POM version specification. While they share the same value (4.0.0), this does not imply a dependency unless one is a parent or module of the other, which isn't indicated here. Maven projects can independently have the same modelVersion without being dependent on each other.",
    "uncertainty": 8,
    "isDependency": False
    }
]


dominant_response = get_dominat_response(responses=responses)

print(dominant_response)


{'plan': "1. Identify the configuration options in question: both are 'project.modelVersion' of type VERSION_NUMBER with value 4.0.0.\n2. Check if these configurations belong to different Maven modules (apollo-audit/apollo-audit-api and apollo-biz).\n3. Determine if there's a parent-child relationship between the POM files or if they are independent.\n4. Assess whether Maven requires modelVersion to be consistent across modules in a multi-module project.\n5. Conclude based on Maven's requirements for modelVersion consistency.", 'rationale': "In Maven, each module's pom.xml must have the same modelVersion as specified in the parent POM if it's part of a multi-module project. If these modules share a common parent with a defined modelVersion, then their individual 'project.modelVersion' values must match to maintain consistency and avoid configuration errors.", 'uncertainty': 8, 'isDependency': True}
